In [3]:
import os
DataPath = './dataset/crop_img/'
# f = open('./img4_path.txt', 'a')
# f1 = open('./img4_path1.txt', 'a')
# f2 = open('./img4_path2.txt', 'a')

cnt = 0
paths = []
for dirs in os.listdir(DataPath):
    path = os.path.join(DataPath, dirs)
    if len(os.listdir(path)) == 0:
        # f.write(path + '\n')
        paths.append(path)
        cnt += 1
print(cnt)

# half = len(paths) // 2
# for path in paths[:half]:
#     f1.write(path + '\n')

# for path in paths[half:]:
#     f2.write(path + '\n')
# f.close()
# f1.close()
# f2.close()

0


In [2]:
DataPath = './dataset/cleansing_data/'
cnt3 = 0
cnt4 = 0
for dirs in os.listdir(DataPath):
    path = os.path.join(DataPath, dirs)
    if len(os.listdir(path)) >= 3:
        cnt3 += 1
    if len(os.listdir(path)) >= 4:
        cnt4 += 1
print('이미지 4개 이상 id: ', cnt4)
print('이미지 3개 이상 id: ', cnt3)

이미지 4개 이상 id:  647
이미지 3개 이상 id:  950


이미지 3개 이상인 id dataset 저장

In [3]:
import cv2

output_path = './dataset/over3_img'

if not os.path.exists(output_path):
    os.mkdir(output_path)

for root, dirs, files in os.walk(DataPath):
    file_id = root.split('/')[-1]
    origin_id_dir = os.path.join(DataPath, file_id)
    change_id_dir = os.path.join(output_path, file_id)

    if len(os.listdir(origin_id_dir)) >= 3:
        if not os.path.exists(change_id_dir):
            os.mkdir(change_id_dir)

        for file in files:
            img_path = os.path.join(root, file)
            save_img_path = os.path.join(change_id_dir, file)
            img = cv2.imread(img_path)
            cv2.imwrite(save_img_path, img)


## Make csv

In [ ]:
txt = open('img4_path_refer.txt', 'r')
refer = {}
while True:
    path = txt.readline()
    if not path:
        break
    path = path.strip().split('G:/crop_img/')[-1].split('/')
    id, name = path[0], path[-1]
    refer[id] = name
txt.close()

print(refer)

In [ ]:
root_path = './dataset/siamese_dataset'
Datapath = root_path + '/train'

id_arr = []
for root, dirs, files in os.walk(Datapath):
    id_arr = dirs
    break

train_refer = {}
for id in id_arr:
    train_refer[id] = refer[id]
print(train_refer)

In [15]:
import cv2
import csv
import random
import itertools


root_path = './dataset/siamese_dataset'
Datapath = root_path + '/train'

id_arr = []
for root, dirs, files in os.walk(Datapath):
    id_arr = dirs
    break

train_refer = {}
for id in id_arr:
    train_refer[id] = refer[id]

f = open('./train_data.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(['img1', 'img2', 'label'])

for ids in os.listdir(Datapath):
    img_id = os.path.join(Datapath, ids)
    images = os.listdir(img_id)

    genuine_pairs = list(itertools.permutations(images, 2))
    num = len(genuine_pairs)
    pair_num = 849 if num > 849 else num
    for genuine in genuine_pairs:
        f, s = genuine
        first = os.path.join(ids, f).replace('\\', '/')
        second = os.path.join(ids, s).replace('\\', '/')
        wr.writerow([first, second, 0])
    
    refer_path = train_refer.pop(ids)
    # 본인을 뺀 나머지 id에서 랜덤으로 imposter id 추출
    join_refer_path = ids + '/' + refer_path
    imposter = random.sample(list(train_refer.values()), pair_num)
    
    for imp in imposter:
        id = imp.split('.')[0]
        imp = id + '/' + imp
        wr.writerow([join_refer_path, imp, 1])
        
    # genuine 개수 1050개 넘어가는 id는 다른 id에서 refer이미지 제외하고 랜덤 추출
    if num > 849:
        diff_num = num - pair_num
        id_idx = id_arr.index(ids)
        my_id = id_arr.pop(id_idx)
        rest_ids = random.sample(id_arr, diff_num)
        
        for rest_id in rest_ids:
            rest = os.path.join(Datapath, rest_id)
            image = os.listdir(rest)
            image.remove(refer[rest_id])
            rand_img = random.choice(image)
            rand_img = rest_id + '/' + rand_img
            wr.writerow([join_refer_path, rand_img, 1])

        id_arr.append(my_id)
    train_refer[ids] = refer_path

f.close()


AttributeError: 'str' object has no attribute 'close'

In [16]:
root_path = './dataset/siamese_dataset'
Datapath = root_path + '/test'

id_arr = []
for root, dirs, files in os.walk(Datapath):
    id_arr = dirs
    break

test_refer = {}
for id in id_arr:
    test_refer[id] = refer[id]

f = open('./test_data.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(['img1', 'img2', 'label'])

for ids in os.listdir(Datapath):
    img_id = os.path.join(Datapath, ids)
    images = os.listdir(img_id)

    genuine_pairs = list(itertools.permutations(images, 2))
    num = len(genuine_pairs)
    pair_num = 199 if num > 199 else num
    for genuine in genuine_pairs:
        f, s = genuine
        first = os.path.join(ids, f).replace('\\', '/')
        second = os.path.join(ids, s).replace('\\', '/')
        wr.writerow([first, second, 0])
    
    refer_path = test_refer.pop(ids)
    # 본인을 뺀 나머지 id에서 랜덤으로 imposter id 추출
    join_refer_path = ids + '/' + refer_path
    imposter = random.sample(list(test_refer.values()), pair_num)
    
    for imp in imposter:
        id = imp.split('.')[0]
        imp = id + '/' + imp
        wr.writerow([join_refer_path, imp, 1])
        
    # genuine 개수 1050개 넘어가는 id는 다른 id에서 refer이미지 제외하고 랜덤 추출
    if num > 199:
        diff_num = num - pair_num
        id_idx = id_arr.index(ids)
        my_id = id_arr.pop(id_idx)
        rest_ids = random.sample(id_arr, diff_num)
        
        for rest_id in rest_ids:
            rest = os.path.join(Datapath, rest_id)
            image = os.listdir(rest)
            image.remove(refer[rest_id])
            rand_img = random.choice(image)
            rand_img = rest_id + '/' + rand_img
            wr.writerow([join_refer_path, rand_img, 1])

        id_arr.append(my_id)
    test_refer[ids] = refer_path

f.close()


AttributeError: 'str' object has no attribute 'close'